# Burgers' Equation and Shock Wave Formation

## Introduction

Burgers' equation is a fundamental partial differential equation that serves as a simplified model for fluid dynamics, nonlinear acoustics, gas dynamics, and traffic flow. It combines nonlinear wave propagation with diffusive effects, making it an ideal testbed for understanding shock wave formation.

## Mathematical Formulation

### The Viscous Burgers' Equation

The one-dimensional viscous Burgers' equation is given by:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}$$

where:
- $u(x, t)$ is the velocity field
- $\nu \geq 0$ is the kinematic viscosity (diffusion coefficient)
- The term $u \frac{\partial u}{\partial x}$ represents nonlinear advection
- The term $\nu \frac{\partial^2 u}{\partial x^2}$ represents viscous diffusion

### The Inviscid Burgers' Equation

When $\nu = 0$, we obtain the inviscid Burgers' equation:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0$$

This can be written in conservative form as:

$$\frac{\partial u}{\partial t} + \frac{\partial}{\partial x}\left(\frac{u^2}{2}\right) = 0$$

## Shock Wave Formation

### Method of Characteristics

For the inviscid equation, the method of characteristics reveals that information propagates along curves where:

$$\frac{dx}{dt} = u$$

Along these characteristics, $u$ remains constant:

$$\frac{du}{dt} = 0$$

This means characteristics are straight lines: $x = x_0 + u_0 t$, where $u_0 = u(x_0, 0)$.

### Breaking Time

When characteristics cross, the solution becomes multi-valued, leading to shock formation. The breaking time $t_b$ is:

$$t_b = \frac{-1}{\min\left(\frac{\partial u_0}{\partial x}\right)}$$

For smooth initial conditions with a negative slope region, shocks form in finite time.

### Rankine-Hugoniot Condition

The shock speed $s$ is determined by the Rankine-Hugoniot jump condition:

$$s = \frac{u_L + u_R}{2}$$

where $u_L$ and $u_R$ are the states to the left and right of the shock.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set up plotting style
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2

## Numerical Methods

### Finite Difference Schemes

We implement several numerical schemes to solve Burgers' equation:

1. **Lax-Friedrichs scheme** (first-order, stable but diffusive):
$$u_j^{n+1} = \frac{1}{2}(u_{j+1}^n + u_{j-1}^n) - \frac{\Delta t}{2\Delta x}\left(\frac{(u_{j+1}^n)^2}{2} - \frac{(u_{j-1}^n)^2}{2}\right)$$

2. **Lax-Wendroff scheme** (second-order):
$$u_j^{n+1} = u_j^n - \frac{\Delta t}{2\Delta x}(f_{j+1}^n - f_{j-1}^n) + \frac{\Delta t^2}{2\Delta x^2}\left(A_{j+1/2}(f_{j+1}^n - f_j^n) - A_{j-1/2}(f_j^n - f_{j-1}^n)\right)$$

where $f = u^2/2$ and $A = u$ is the flux Jacobian.

3. **Godunov scheme** with exact Riemann solver (upwind, entropy-satisfying)

In [ ]:
def flux(u):
    """Burgers flux function f(u) = u^2/2"""
    return 0.5 * u**2

def godunov_flux(u_left, u_right):
    """
    Godunov numerical flux for Burgers' equation.
    Uses exact Riemann solver.
    """
    if u_left >= u_right:
        # Shock wave
        if u_left + u_right >= 0:
            return flux(u_left)
        else:
            return flux(u_right)
    else:
        # Rarefaction wave
        if u_left >= 0:
            return flux(u_left)
        elif u_right <= 0:
            return flux(u_right)
        else:
            return 0.0  # Sonic point

def lax_friedrichs(u, dt, dx):
    """Lax-Friedrichs scheme for Burgers' equation."""
    n = len(u)
    u_new = np.zeros_like(u)
    
    for j in range(1, n-1):
        u_new[j] = 0.5 * (u[j+1] + u[j-1]) - (dt / (2*dx)) * (flux(u[j+1]) - flux(u[j-1]))
    
    # Boundary conditions (outflow)
    u_new[0] = u_new[1]
    u_new[-1] = u_new[-2]
    
    return u_new

def godunov_scheme(u, dt, dx):
    """Godunov scheme for Burgers' equation."""
    n = len(u)
    u_new = np.zeros_like(u)
    
    # Compute numerical fluxes at cell interfaces
    f_half = np.zeros(n+1)
    for j in range(n-1):
        f_half[j+1] = godunov_flux(u[j], u[j+1])
    
    # Update interior cells
    for j in range(1, n-1):
        u_new[j] = u[j] - (dt/dx) * (f_half[j+1] - f_half[j])
    
    # Boundary conditions
    u_new[0] = u_new[1]
    u_new[-1] = u_new[-2]
    
    return u_new

## Simulation: Shock Formation from Smooth Initial Data

We demonstrate shock formation using a sinusoidal initial condition:

$$u_0(x) = \sin(\pi x)$$

on the domain $x \in [-1, 1]$. The breaking time for this initial condition is:

$$t_b = \frac{1}{\pi}$$

since $\min(\partial u_0/\partial x) = -\pi$.

In [ ]:
# Domain and grid parameters
x_min, x_max = -1.0, 1.0
nx = 400
dx = (x_max - x_min) / (nx - 1)
x = np.linspace(x_min, x_max, nx)

# Time parameters
t_final = 0.5
cfl = 0.5  # CFL number for stability

# Initial condition: sine wave
u0 = np.sin(np.pi * x)

# Theoretical breaking time
t_break = 1.0 / np.pi
print(f"Theoretical breaking time: t_b = 1/π ≈ {t_break:.4f}")
print(f"Simulation end time: t = {t_final}")
print(f"Number of grid points: {nx}")
print(f"Grid spacing: Δx = {dx:.6f}")

In [ ]:
def solve_burgers(u0, x, t_final, cfl, scheme='godunov'):
    """
    Solve Burgers' equation with given initial condition.
    
    Parameters:
    -----------
    u0 : array
        Initial condition
    x : array
        Spatial grid
    t_final : float
        Final simulation time
    cfl : float
        CFL number
    scheme : str
        'godunov' or 'lax_friedrichs'
    
    Returns:
    --------
    times : list
        Time values
    solutions : list
        Solution arrays at each time
    """
    dx = x[1] - x[0]
    u = u0.copy()
    t = 0.0
    
    times = [0.0]
    solutions = [u0.copy()]
    
    # Select scheme
    if scheme == 'godunov':
        update = godunov_scheme
    else:
        update = lax_friedrichs
    
    while t < t_final:
        # Adaptive time step based on CFL condition
        u_max = np.max(np.abs(u))
        if u_max > 1e-10:
            dt = cfl * dx / u_max
        else:
            dt = cfl * dx
        
        # Don't exceed final time
        if t + dt > t_final:
            dt = t_final - t
        
        # Update solution
        u = update(u, dt, dx)
        t += dt
        
        # Store solution at regular intervals
        if len(times) < 100 or t >= t_final:
            times.append(t)
            solutions.append(u.copy())
    
    return times, solutions

# Solve with Godunov scheme
times, solutions = solve_burgers(u0, x, t_final, cfl, scheme='godunov')
print(f"Simulation completed with {len(times)} time steps")

## Visualization of Shock Formation

We plot the solution at several time instances to observe the steepening of the wave profile and eventual shock formation.

In [ ]:
# Select time instances for plotting
plot_times = [0.0, 0.1, 0.2, t_break, 0.4, t_final]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, t_plot in enumerate(plot_times):
    # Find closest time in solution
    time_idx = np.argmin(np.abs(np.array(times) - t_plot))
    u_plot = solutions[time_idx]
    actual_time = times[time_idx]
    
    ax = axes[idx]
    ax.plot(x, u_plot, 'b-', linewidth=2)
    ax.plot(x, u0, 'k--', alpha=0.3, label='Initial')
    
    # Mark shock region if past breaking time
    if actual_time > t_break:
        ax.axvline(x=0, color='r', linestyle=':', alpha=0.5, label='Shock location')
    
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x, t)$')
    ax.set_title(f'$t = {actual_time:.3f}$' + 
                 (' (breaking time)' if abs(actual_time - t_break) < 0.02 else ''))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([-1.5, 1.5])
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right', fontsize=8)

plt.suptitle("Burgers' Equation: Shock Wave Formation from Sinusoidal Initial Data", 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Method of Characteristics Visualization

To better understand shock formation, we visualize the characteristic curves in the $x$-$t$ plane. The crossing of characteristics indicates shock formation.

In [ ]:
# Plot characteristic curves
fig, ax = plt.subplots(figsize=(10, 8))

# Initial positions for characteristics
x0_chars = np.linspace(-0.9, 0.9, 30)
t_char = np.linspace(0, t_final, 100)

for x0 in x0_chars:
    u0_char = np.sin(np.pi * x0)  # Initial velocity at this point
    x_char = x0 + u0_char * t_char  # Characteristic line
    
    # Color based on initial velocity
    color = plt.cm.coolwarm((u0_char + 1) / 2)
    ax.plot(x_char, t_char, color=color, alpha=0.7, linewidth=1)

# Mark breaking time
ax.axhline(y=t_break, color='k', linestyle='--', linewidth=2, label=f'Breaking time $t_b = 1/\\pi$')

ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$t$', fontsize=12)
ax.set_title('Characteristic Curves for Burgers\' Equation', fontsize=14)
ax.set_xlim([-2, 2])
ax.set_ylim([0, t_final])
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=plt.cm.coolwarm, norm=plt.Normalize(-1, 1))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('Initial velocity $u_0$', fontsize=10)

plt.tight_layout()
plt.show()

## Effect of Viscosity

Now we compare the inviscid solution with viscous solutions to see how diffusion smooths the shock.

In [ ]:
def solve_viscous_burgers(u0, x, t_final, nu, dt_factor=0.4):
    """
    Solve viscous Burgers' equation using FTCS for diffusion
    and upwind for advection.
    """
    dx = x[1] - x[0]
    u = u0.copy()
    t = 0.0
    
    # Stability constraint for viscous term
    dt_visc = dt_factor * dx**2 / (2 * nu) if nu > 0 else np.inf
    
    while t < t_final:
        # CFL for advection
        u_max = np.max(np.abs(u))
        dt_adv = dt_factor * dx / u_max if u_max > 1e-10 else dt_factor * dx
        
        # Use minimum for stability
        dt = min(dt_adv, dt_visc)
        if t + dt > t_final:
            dt = t_final - t
        
        # Godunov for advection + central difference for diffusion
        u_new = godunov_scheme(u, dt, dx)
        
        # Add viscous term
        if nu > 0:
            for j in range(1, len(u)-1):
                u_new[j] += nu * dt / dx**2 * (u[j+1] - 2*u[j] + u[j-1])
        
        u = u_new
        t += dt
    
    return u

# Compare solutions with different viscosities
viscosities = [0.0, 0.005, 0.02, 0.05]
t_compare = 0.4

fig, ax = plt.subplots(figsize=(10, 6))

for nu in viscosities:
    if nu == 0:
        # Use inviscid solution
        time_idx = np.argmin(np.abs(np.array(times) - t_compare))
        u_final = solutions[time_idx]
        label = f'$\\nu = 0$ (inviscid)'
    else:
        u_final = solve_viscous_burgers(u0, x, t_compare, nu)
        label = f'$\\nu = {nu}$'
    
    ax.plot(x, u_final, linewidth=2, label=label)

ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$u(x, t)$', fontsize=12)
ax.set_title(f'Effect of Viscosity on Shock Structure at $t = {t_compare}$', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_xlim([x_min, x_max])
ax.set_ylim([-1.5, 1.5])

plt.tight_layout()
plt.show()

## Riemann Problem: Shock and Rarefaction Waves

The Riemann problem is an initial value problem with piecewise constant initial data:

$$u_0(x) = \begin{cases} u_L & x < 0 \\ u_R & x > 0 \end{cases}$$

The exact solution depends on the relationship between $u_L$ and $u_R$:

1. **Shock wave** ($u_L > u_R$): A discontinuity traveling at speed $s = (u_L + u_R)/2$
2. **Rarefaction wave** ($u_L < u_R$): A smooth fan connecting the two states

In [ ]:
def exact_riemann_solution(x, t, u_L, u_R):
    """
    Exact solution to the Riemann problem for Burgers' equation.
    """
    if t == 0:
        return np.where(x < 0, u_L, u_R)
    
    u = np.zeros_like(x)
    xi = x / t  # Similarity variable
    
    if u_L > u_R:
        # Shock wave
        s = 0.5 * (u_L + u_R)
        u = np.where(xi < s, u_L, u_R)
    else:
        # Rarefaction wave
        u = np.where(xi <= u_L, u_L, 
                     np.where(xi >= u_R, u_R, xi))
    
    return u

# Set up Riemann problems
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Case 1: Shock wave (u_L > u_R)
u_L, u_R = 1.0, 0.0
x_riemann = np.linspace(-1, 1, 400)

ax = axes[0]
for t in [0.0, 0.2, 0.4, 0.6]:
    u_exact = exact_riemann_solution(x_riemann, t, u_L, u_R)
    ax.plot(x_riemann, u_exact, label=f'$t = {t}$', linewidth=2)

ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$u(x, t)$', fontsize=12)
ax.set_title(f'Shock Wave: $u_L = {u_L}$, $u_R = {u_R}$', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim([-0.2, 1.2])

# Case 2: Rarefaction wave (u_L < u_R)
u_L, u_R = -0.5, 1.0

ax = axes[1]
for t in [0.0, 0.2, 0.4, 0.6]:
    u_exact = exact_riemann_solution(x_riemann, t, u_L, u_R)
    ax.plot(x_riemann, u_exact, label=f'$t = {t}$', linewidth=2)

ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$u(x, t)$', fontsize=12)
ax.set_title(f'Rarefaction Wave: $u_L = {u_L}$, $u_R = {u_R}$', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim([-0.7, 1.2])

plt.tight_layout()
plt.show()

## Numerical Verification: Convergence Study

We verify our numerical scheme by performing a convergence study. For smooth solutions (before shock formation), we expect second-order accuracy; after shock formation, the scheme reduces to first-order at the shock.

In [ ]:
# Convergence study using Riemann problem with exact solution
u_L, u_R = 1.0, 0.0
t_test = 0.3
grid_sizes = [50, 100, 200, 400, 800]
errors = []

for nx in grid_sizes:
    x_test = np.linspace(-1, 1, nx)
    dx = x_test[1] - x_test[0]
    
    # Initial condition (step function)
    u0_test = np.where(x_test < 0, u_L, u_R)
    
    # Solve numerically
    _, sols = solve_burgers(u0_test, x_test, t_test, cfl=0.5, scheme='godunov')
    u_num = sols[-1]
    
    # Exact solution
    u_exact = exact_riemann_solution(x_test, t_test, u_L, u_R)
    
    # L1 error
    error = np.sum(np.abs(u_num - u_exact)) * dx
    errors.append(error)

# Plot convergence
fig, ax = plt.subplots(figsize=(8, 6))

dx_values = [2.0/n for n in grid_sizes]
ax.loglog(dx_values, errors, 'bo-', linewidth=2, markersize=8, label='Godunov scheme')

# Reference lines
dx_ref = np.array(dx_values)
ax.loglog(dx_ref, 0.5*dx_ref, 'k--', label='$O(\\Delta x)$')
ax.loglog(dx_ref, 2*dx_ref**0.5, 'k:', label='$O(\\Delta x^{1/2})$')

ax.set_xlabel('$\\Delta x$', fontsize=12)
ax.set_ylabel('$L^1$ Error', fontsize=12)
ax.set_title('Convergence Study: Riemann Problem', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Calculate convergence rate
rates = [np.log(errors[i]/errors[i+1])/np.log(2) for i in range(len(errors)-1)]
print("Grid sizes:", grid_sizes)
print("L1 errors:", [f"{e:.6f}" for e in errors])
print("Convergence rates:", [f"{r:.2f}" for r in rates])

plt.tight_layout()
plt.show()

## Conclusions

This notebook demonstrated the fundamental properties of Burgers' equation and shock wave formation:

1. **Nonlinear steepening**: The advection term $u\partial_x u$ causes wave steepening, where faster-moving parts of the wave catch up with slower parts.

2. **Shock formation**: For inviscid flow, smooth initial data develops discontinuities (shocks) in finite time. The breaking time can be predicted from the initial velocity gradient.

3. **Entropy condition**: Physical shocks must satisfy the entropy condition $u_L > u_R$, ensuring that characteristics enter the shock from both sides.

4. **Viscous smoothing**: Small viscosity regularizes shocks into thin transition layers, with the shock width scaling as $O(\nu/|u_L - u_R|)$.

5. **Numerical methods**: Conservative finite difference schemes like Godunov's method correctly capture shock waves without spurious oscillations.

Burgers' equation serves as an essential stepping stone for understanding more complex hyperbolic conservation laws, including the Euler equations of gas dynamics and the shallow water equations.